### Chapter 26: SystemVerilog for Synthesis

#### Introduction

Synthesis is the process of converting RTL (Register Transfer Level) code into a gate-level netlist that can be implemented in hardware. Understanding which SystemVerilog constructs are synthesizable and how to write efficient RTL code is crucial for successful digital design. This chapter covers the essential aspects of writing synthesizable SystemVerilog code.

#### Synthesizable vs. Non-Synthesizable Constructs

##### Synthesizable Constructs

**Basic Data Types:**
```systemverilog
// Synthesizable data types
logic [31:0] data_reg;
logic        enable;
logic [7:0]  counter;
bit   [15:0] address;

// Arrays (with limitations)
logic [7:0] memory [0:255];  // Small memories
logic [3:0] lookup_table [0:15];
```

**Always Blocks:**
```systemverilog
// Synthesizable always blocks
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        counter <= 8'b0;
    else if (enable)
        counter <= counter + 1;
end

always_comb begin
    sum = a + b;
    carry_out = (a & b) | ((a ^ b) & carry_in);
end
```

**Control Structures:**
```systemverilog
// If-else statements
always_comb begin
    if (select == 2'b00)
        mux_out = input_a;
    else if (select == 2'b01)
        mux_out = input_b;
    else if (select == 2'b10)
        mux_out = input_c;
    else
        mux_out = input_d;
end

// Case statements
always_comb begin
    case (opcode)
        3'b000: alu_out = a + b;
        3'b001: alu_out = a - b;
        3'b010: alu_out = a & b;
        3'b011: alu_out = a | b;
        default: alu_out = 32'b0;
    endcase
end

// For loops (with constant bounds)
always_comb begin
    parity = 1'b0;
    for (int i = 0; i < 32; i++) begin
        parity = parity ^ data[i];
    end
end
```

##### Non-Synthesizable Constructs

**Timing Control:**
```systemverilog
// Non-synthesizable - delays
#10 data = new_value;
data <= #5 new_value;

// Non-synthesizable - wait statements
wait (ready == 1'b1);
@(posedge clk);  // Outside of always blocks
```

**System Tasks and Functions:**
```systemverilog
// Non-synthesizable system tasks
$display("Value = %d", data);
$monitor("clock = %b", clk);
$random();
$time;
$finish;
```

**Advanced Data Types:**
```systemverilog
// Non-synthesizable constructs
real    floating_point;
string  text_data;
class   my_class;
mailbox data_mailbox;
semaphore resource_sem;

// Dynamic arrays
int dynamic_array[];
```

**File I/O:**
```systemverilog
// Non-synthesizable file operations
int file_handle;
file_handle = $fopen("data.txt", "r");
$fread(data, file_handle);
$fclose(file_handle);
```

#### RTL Coding Guidelines

##### Clock and Reset Guidelines

**Single Clock Domain:**
```systemverilog
// Good practice - single clock domain
module counter (
    input  logic        clk,
    input  logic        rst_n,
    input  logic        enable,
    output logic [7:0]  count
);

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        count <= 8'b0;
    else if (enable)
        count <= count + 1;
end

endmodule
```

**Proper Reset Usage:**
```systemverilog
// Asynchronous reset, synchronous deassertion
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        // Reset all registers
        state <= IDLE;
        counter <= 8'b0;
        data_valid <= 1'b0;
    end else begin
        // Normal operation
        case (state)
            IDLE: if (start) state <= ACTIVE;
            ACTIVE: begin
                counter <= counter + 1;
                if (counter == 8'hFF)
                    state <= DONE;
            end
            DONE: if (ack) state <= IDLE;
        endcase
    end
end
```

##### Combinational Logic Guidelines

**Avoid Latches:**
```systemverilog
// Bad - creates latches
always_comb begin
    if (enable)
        output_data = input_data;
    // Missing else clause creates latch
end

// Good - no latches
always_comb begin
    if (enable)
        output_data = input_data;
    else
        output_data = previous_data;
end

// Better - default assignment
always_comb begin
    output_data = previous_data;  // Default
    if (enable)
        output_data = input_data;
end
```

**Complete Case Statements:**
```systemverilog
// Good practice with default
always_comb begin
    case (state)
        2'b00: next_state = 2'b01;
        2'b01: next_state = 2'b10;
        2'b10: next_state = 2'b11;
        2'b11: next_state = 2'b00;
        default: next_state = 2'b00;  // Prevents latches
    endcase
end

// Using unique case for synthesis optimization
always_comb begin
    unique case (state)
        2'b00: next_state = 2'b01;
        2'b01: next_state = 2'b10;
        2'b10: next_state = 2'b11;
        2'b11: next_state = 2'b00;
    endcase
end
```

##### State Machine Coding

**Template for FSM:**
```systemverilog
typedef enum logic [2:0] {
    IDLE    = 3'b000,
    START   = 3'b001,
    PROCESS = 3'b010,
    WAIT    = 3'b011,
    DONE    = 3'b100
} state_t;

module fsm_example (
    input  logic    clk,
    input  logic    rst_n,
    input  logic    start,
    input  logic    data_ready,
    output logic    busy,
    output logic    complete
);

state_t current_state, next_state;

// State register
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        current_state <= IDLE;
    else
        current_state <= next_state;
end

// Next state logic
always_comb begin
    next_state = current_state;  // Default assignment
    case (current_state)
        IDLE: 
            if (start) next_state = START;
        START: 
            next_state = PROCESS;
        PROCESS: 
            if (data_ready) next_state = WAIT;
        WAIT: 
            next_state = DONE;
        DONE: 
            next_state = IDLE;
        default: 
            next_state = IDLE;
    endcase
end

// Output logic
always_comb begin
    busy = (current_state != IDLE) && (current_state != DONE);
    complete = (current_state == DONE);
end

endmodule
```

#### Timing and Area Considerations

##### Critical Path Optimization

**Pipeline Insertion:**
```systemverilog
// Non-pipelined - long critical path
module multiplier_comb (
    input  logic [15:0] a, b,
    output logic [31:0] product
);

always_comb begin
    product = a * b;  // Long combinational path
end

endmodule

// Pipelined version - shorter critical path
module multiplier_pipe (
    input  logic        clk,
    input  logic        rst_n,
    input  logic [15:0] a, b,
    output logic [31:0] product
);

logic [15:0] a_reg, b_reg;
logic [31:0] mult_result;

// Input registers
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        a_reg <= 16'b0;
        b_reg <= 16'b0;
    end else begin
        a_reg <= a;
        b_reg <= b;
    end
end

// Multiplication
always_comb begin
    mult_result = a_reg * b_reg;
end

// Output register
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        product <= 32'b0;
    else
        product <= mult_result;
end

endmodule
```

**Operator Inference:**
```systemverilog
// Synthesis tools infer appropriate operators
module arithmetic_units (
    input  logic        clk,
    input  logic [31:0] a, b,
    output logic [31:0] sum,
    output logic [31:0] difference,
    output logic [63:0] product,
    output logic [31:0] quotient
);

always_ff @(posedge clk) begin
    sum <= a + b;        // Adder
    difference <= a - b;  // Subtractor
    product <= a * b;    // Multiplier
    quotient <= a / b;   // Divider (use carefully)
end

endmodule
```

##### Resource Sharing

**Manual Resource Sharing:**
```systemverilog
module shared_adder (
    input  logic        clk,
    input  logic        rst_n,
    input  logic        sel,
    input  logic [31:0] a, b, c, d,
    output logic [31:0] result
);

logic [31:0] operand1, operand2;

// Multiplexer for resource sharing
always_comb begin
    if (sel) begin
        operand1 = a;
        operand2 = b;
    end else begin
        operand1 = c;
        operand2 = d;
    end
end

// Shared adder
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        result <= 32'b0;
    else
        result <= operand1 + operand2;
end

endmodule
```

##### Memory Inference

**BRAM Inference:**
```systemverilog
module inferred_bram (
    input  logic        clk,
    input  logic        we,
    input  logic [9:0]  addr,
    input  logic [31:0] din,
    output logic [31:0] dout
);

logic [31:0] memory [0:1023];

always_ff @(posedge clk) begin
    if (we)
        memory[addr] <= din;
    dout <= memory[addr];
end

endmodule
```

**Distributed RAM Inference:**
```systemverilog
module inferred_dist_ram (
    input  logic       clk,
    input  logic       we,
    input  logic [7:0] addr,
    input  logic [7:0] din,
    output logic [7:0] dout
);

logic [7:0] memory [0:255];

always_ff @(posedge clk) begin
    if (we)
        memory[addr] <= din;
end

assign dout = memory[addr];  // Asynchronous read

endmodule
```

#### Synthesis Tool Considerations

##### Synthesis Directives

**Synthesis Attributes:**
```systemverilog
module synthesis_attributes (
    input  logic        clk,
    input  logic [31:0] data_in,
    output logic [31:0] data_out
);

// Keep intermediate signals for debugging
(* keep = "true" *) logic [31:0] intermediate;

// Don't touch this logic during optimization
(* dont_touch = "true" *) logic important_signal;

// RAM style specification
(* ram_style = "block" *) logic [31:0] block_ram [0:1023];
(* ram_style = "distributed" *) logic [7:0] dist_ram [0:255];

// FSM encoding
typedef enum logic [2:0] {
    STATE_A = 3'b001,
    STATE_B = 3'b010,
    STATE_C = 3'b100
} (* fsm_encoding = "one_hot" *) state_enum_t;

endmodule
```

##### Synthesis Pragmas

**Tool-Specific Directives:**
```systemverilog
module pragma_examples (
    input  logic        clk,
    input  logic        rst_n,
    input  logic [31:0] a, b,
    output logic [31:0] result
);

// Xilinx-specific pragmas
// synthesis translate_off
initial begin
    $display("This code is ignored during synthesis");
end
// synthesis translate_on

// Parallel case directive
always_comb begin
    // synthesis parallel_case
    case (select)
        2'b00: result = a;
        2'b01: result = b;
        2'b10: result = a + b;
        2'b11: result = a - b;
    endcase
end

// Full case directive
always_comb begin
    // synthesis full_case
    case (mode)
        2'b00: operation = ADD;
        2'b01: operation = SUB;
        2'b10: operation = AND;
        2'b11: operation = OR;
    endcase
end

endmodule
```

##### Design for Testability

**Scan Chain Insertion:**
```systemverilog
module testable_design (
    input  logic       clk,
    input  logic       rst_n,
    input  logic       scan_enable,
    input  logic       scan_in,
    output logic       scan_out,
    input  logic [7:0] data_in,
    output logic [7:0] data_out
);

logic [7:0] reg_chain [0:3];

genvar i;
generate
    for (i = 0; i < 4; i++) begin : reg_gen
        always_ff @(posedge clk or negedge rst_n) begin
            if (!rst_n)
                reg_chain[i] <= 8'b0;
            else if (scan_enable)
                reg_chain[i] <= (i == 0) ? {7'b0, scan_in} : 
                                reg_chain[i-1];
            else
                reg_chain[i] <= (i == 0) ? data_in : 
                                reg_chain[i-1];
        end
    end
endgenerate

assign data_out = reg_chain[3];
assign scan_out = reg_chain[3][7];

endmodule
```

#### Best Practices for Synthesis

##### Code Structure

**Hierarchical Design:**
```systemverilog
// Top-level module
module processor_top (
    input  logic        clk,
    input  logic        rst_n,
    input  logic [31:0] instruction,
    output logic [31:0] result
);

// Instantiate submodules
alu u_alu (
    .clk(clk),
    .rst_n(rst_n),
    .a(operand_a),
    .b(operand_b),
    .op(alu_op),
    .result(alu_result)
);

register_file u_regfile (
    .clk(clk),
    .rst_n(rst_n),
    .read_addr1(rs1),
    .read_addr2(rs2),
    .write_addr(rd),
    .write_data(write_data),
    .write_enable(reg_we),
    .read_data1(operand_a),
    .read_data2(operand_b)
);

control_unit u_control (
    .instruction(instruction),
    .alu_op(alu_op),
    .reg_we(reg_we),
    .rs1(rs1),
    .rs2(rs2),
    .rd(rd)
);

endmodule
```

##### Naming Conventions

```systemverilog
module naming_example (
    // Clock and reset
    input  logic        clk,
    input  logic        rst_n,     // Active low reset
    
    // Control signals
    input  logic        enable_i,  // Input enable
    output logic        valid_o,   // Output valid
    output logic        ready_o,   // Output ready
    
    // Data signals
    input  logic [31:0] data_i,    // Input data
    output logic [31:0] data_o,    // Output data
    
    // Internal signals use descriptive names
    logic [31:0] processed_data;
    logic        processing_complete;
    logic [3:0]  state_counter;
);
```

##### Performance Optimization

**Clock Gating:**
```systemverilog
module clock_gated_register (
    input  logic        clk,
    input  logic        rst_n,
    input  logic        enable,
    input  logic [31:0] data_in,
    output logic [31:0] data_out
);

logic gated_clk;

// Clock gating cell (tool-specific)
// Usually inferred by synthesis tools
assign gated_clk = clk & enable;

always_ff @(posedge gated_clk or negedge rst_n) begin
    if (!rst_n)
        data_out <= 32'b0;
    else
        data_out <= data_in;
end

endmodule
```

#### Common Synthesis Issues and Solutions

##### Timing Issues

**Setup Time Violations:**
```systemverilog
// Problem: Long combinational path
always_ff @(posedge clk) begin
    result <= ((a + b) * c) - (d & e);  // Long path
end

// Solution: Pipeline the operation
logic [31:0] stage1_add, stage1_and;
logic [31:0] stage2_mult;

always_ff @(posedge clk) begin
    // Stage 1
    stage1_add <= a + b;
    stage1_and <= d & e;
    
    // Stage 2
    stage2_mult <= stage1_add * c;
    
    // Stage 3
    result <= stage2_mult - stage1_and;
end
```

##### Area Issues

**Reducing Logic Usage:**
```systemverilog
// Inefficient: Multiple comparators
always_comb begin
    if (data == 8'h00 || data == 8'h01 || data == 8'h02 || data == 8'h03)
        category = 2'b00;
    else if (data == 8'h04 || data == 8'h05 || data == 8'h06 || data == 8'h07)
        category = 2'b01;
    // ... more conditions
end

// Efficient: Use bit slicing
always_comb begin
    case (data[7:2])
        6'b000000: category = 2'b00;  // 0x00-0x03
        6'b000001: category = 2'b01;  // 0x04-0x07
        6'b000010: category = 2'b10;  // 0x08-0x0B
        default:   category = 2'b11;
    endcase
end
```

#### Summary

Writing synthesizable SystemVerilog code requires understanding the distinction between behavioral and structural descriptions. Key guidelines include using proper clocking and reset strategies, avoiding latches through complete case coverage, implementing efficient state machines, and considering timing and area constraints. Synthesis tools provide various optimization opportunities through proper coding practices and strategic use of synthesis directives. Following these guidelines ensures that your SystemVerilog designs can be successfully synthesized into efficient hardware implementations.